## Neural Network

In [248]:
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow.keras.backend as K
from tqdm import tqdm
import seaborn as sns

In [249]:
#Amount of features inputted in NN
NUMBER_FEATURES = 9

#Gaussian Noise
USE_GAUSSIAN_NOISE = False
VALUE_GAUSSIAN_NOISE = 70 # Only needed when Gaussian noise = True

#Activation function to use
ACTIVATION_FUNCTION = 'relu'

#Train length
EPOCHS = 10000
STEPS_PER_EPOCH = 10

#Learning rate
MAX_LEARNING_RATE = 1e-4
COSINE_CYCLES = 100

config = dict(NUMBER_FEATURES = NUMBER_FEATURES, USE_GAUSSIAN_NOISE = USE_GAUSSIAN_NOISE, 
              VALUE_GAUSSIAN_NOISE = VALUE_GAUSSIAN_NOISE, ACTIVATION_FUNCTION = ACTIVATION_FUNCTION,
              EPOCHS = EPOCHS, STEPS_PER_EPOCH = STEPS_PER_EPOCH, MAX_LEARNING_RATE = MAX_LEARNING_RATE,
              COSINE_CYCLES = COSINE_CYCLES)

In [250]:
use_gaussian_noise = config["USE_GAUSSIAN_NOISE"]
value_gaussian_noise = config["VALUE_GAUSSIAN_NOISE"]

if use_gaussian_noise:
    inp = tf.keras.layers.GaussianNoise(value_gaussian_noise)(inp)

In [251]:
def build_model(config):
    size = config["NUMBER_FEATURES"]
    actfunc = config["ACTIVATION_FUNCTION"]
    if(actfunc == 'swish'):
        actfunc = tf.keras.activations.swish

    inp = tf.keras.layers.Input(shape=(1,size), name = "input_features")
    
    inp2 = tf.keras.layers.Input(shape = (1,1), name = "FVC_Start_Weeks_from_start")
    
    inputs = [inp]
    outputs = []
    
    x = tf.keras.layers.Dense(100, activation=actfunc)(inp)
    x = tf.keras.layers.Dense(75, activation=actfunc)(x)
    x = tf.keras.layers.Dense(50, activation=actfunc)(x)
    x = tf.keras.layers.Dense(25, activation=actfunc)(x)
    x = tf.keras.layers.Dense(15, activation=actfunc)(x)
    x = tf.keras.layers.Dense(10, activation=actfunc)(x)

    
    # output : [slope, s, FVC_start, weeks_from_start]
    outputs += [tf.keras.layers.Dense(2, name = "Output_a_s")(x)]

    model = tf.keras.Model(inputs = inputs, outputs = outputs)
    
    def Laplace_log_likelihood(y_true, y_pred):
        # y_pred = [slope, s, FVC_start, weeks_from_start]
        tf.dtypes.cast(y_true, tf.float32)
        tf.dtypes.cast(y_pred, tf.float32)
        
        #slope = y_pred[:,0]
        #s = y_pred[:,1]
        
        FVC_pred = y_pred[:,0]
        sigma = y_pred[:,1]
        
        FVC_now = y_true[:,0]
        weeks_from_start = y_true[:,1]
        FVC_start = y_true[:,2]
        
        #sigma = s * weeks_from_start
        
        # Kan probleem worden by ReLu omdat slope negatief wordt door minimalisering Loss
        #FVC_pred = weeks_from_start * slope + FVC_start

        ## ** Hier kan een fout komen doordat de afgeleide moeilijker te berekenen is
        sigma_clip = tf.maximum(tf.abs(sigma), 70)*2
        delta = tf.abs(FVC_now - FVC_pred)
        delta = tf.minimum(delta, 1000)
        ## **
        
        sq2 = tf.sqrt(tf.dtypes.cast(2, dtype=tf.float32))
        loss = (delta / sigma_clip)*sq2 + tf.math.log(sigma_clip * sq2)
        return K.mean(loss)
    
    opt = tf.keras.optimizers.Adam()
    
    model.compile(optimizer=opt, loss=Laplace_log_likelihood)#, loss_weights = [...])

    return model

In [252]:
def get_cosine_annealing_lr_callback(lr_max=1e-4, n_epochs=10000, n_cycles=100):
    epochs_per_cycle = np.floor(n_epochs / n_cycles)

    def lrfn(epoch):
        cos_inner = (np.pi * (epoch % epochs_per_cycle)) / epochs_per_cycle
        return lr_max / 2 * (np.cos(cos_inner) + 1)

    lr_callback = tf.keras.callbacks.LearningRateScheduler(lrfn, verbose=False)
    return lr_callback

In [253]:
model = build_model(config)
#tf.keras.utils.plot_model(model)
model.summary()

Model: "functional_47"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_features (InputLayer)  [(None, 1, 9)]            0         
_________________________________________________________________
dense_138 (Dense)            (None, 1, 100)            1000      
_________________________________________________________________
dense_139 (Dense)            (None, 1, 75)             7575      
_________________________________________________________________
dense_140 (Dense)            (None, 1, 50)             3800      
_________________________________________________________________
dense_141 (Dense)            (None, 1, 25)             1275      
_________________________________________________________________
dense_142 (Dense)            (None, 1, 15)             390       
_________________________________________________________________
dense_143 (Dense)            (None, 1, 10)           

## Preparing the Data

In [254]:
## !!!! https://stanford.edu/~shervine/blog/keras-how-to-generate-data-on-the-fly

# Dit is voor data generation on the fly voor bijv gaussian noise

In [255]:
df = pd.read_csv('../input/osic-pulmonary-fibrosis-progression/train.csv')
alldatapoints = []
for patient in df.Patient.unique():
    patientperweek = []
    weeks = df.loc[df.Patient == patient]['Weeks']
    for week1 in weeks:
        dfnew = df.loc[(df.Patient == patient) & (df.Weeks != week1)]
        dfnew = dfnew.assign(Targetweek = week1)
        dfnew = dfnew.assign(TargetFVC = df.loc[(df.Patient == patient)&(df.Weeks == week1)]['FVC'].values[0])
        patientperweek.append(dfnew)
    alldatapoints.append(pd.concat(patientperweek))

train = pd.DataFrame(pd.concat(alldatapoints))    
train["Sex"] = (train['Sex']=="Male").astype(int)
train = pd.concat([train,pd.get_dummies(train['SmokingStatus'])],axis = 1).reset_index(drop = True)
labels = pd.DataFrame(train[["TargetFVC","Targetweek","FVC"]])
labels = labels.astype("float32")
train = train.drop(["SmokingStatus", "TargetFVC", "Patient"],axis = 1)
for i in range(len(train)):
    train.loc[i, "Targetweek"] = train.loc[i, "Targetweek"] - train.loc[i, "Weeks"]

In [256]:
data = {"input_features": train[["Weeks", "FVC", "Percent", "Age", "Sex", 
                                 "Currently smokes", "Ex-smoker", "Never smoked", "Targetweek"]]
        , "FVC_Start_Weeks_from_start": train["Targetweek"]}

In [257]:
# from sklearn.model_selection import train_test_split
# x_train, x_valid, y_train, y_valid = train_test_split(data["input_features"], labels, test_size=0.33, shuffle= True)

# Hierbegint nieuwe patient dus soort van stratified
x_train = data["input_features"][:10004]
y_train = labels[:10004]

x_val = data["input_features"][10004:]
y_val = labels[10004:]

In [258]:
lr_cb = get_cosine_annealing_lr_callback(lr_max=config["MAX_LEARNING_RATE"], n_epochs=config["EPOCHS"], n_cycles=config["COSINE_CYCLES"])

In [ ]:
model.fit(x_train, 
          y_train, validation_data = (x_val,y_val),epochs = 10000,steps_per_epoch =10,
          verbose = 2, callbacks = [lr_cb])

Epoch 1/10000
10/10 - 0s - loss: 11.9456 - val_loss: 11.4215
Epoch 2/10000
10/10 - 0s - loss: 11.0919 - val_loss: 10.5618
Epoch 3/10000
10/10 - 0s - loss: 10.3734 - val_loss: 9.9808
Epoch 4/10000
10/10 - 0s - loss: 9.8769 - val_loss: 9.5873
Epoch 5/10000
10/10 - 0s - loss: 9.5484 - val_loss: 9.3504
Epoch 6/10000
10/10 - 0s - loss: 9.3444 - val_loss: 9.1936
Epoch 7/10000
10/10 - 0s - loss: 9.2014 - val_loss: 9.0840
Epoch 8/10000
10/10 - 0s - loss: 9.0969 - val_loss: 9.0010
Epoch 9/10000
10/10 - 0s - loss: 9.0166 - val_loss: 8.9348
Epoch 10/10000
10/10 - 0s - loss: 8.9525 - val_loss: 8.8820
Epoch 11/10000
10/10 - 0s - loss: 8.9006 - val_loss: 8.8392
Epoch 12/10000
10/10 - 0s - loss: 8.8582 - val_loss: 8.8042
Epoch 13/10000
10/10 - 0s - loss: 8.8234 - val_loss: 8.7755
Epoch 14/10000
10/10 - 0s - loss: 8.7945 - val_loss: 8.7518
Epoch 15/10000
10/10 - 0s - loss: 8.7706 - val_loss: 8.7324
Epoch 16/10000
10/10 - 0s - loss: 8.7508 - val_loss: 8.7165
Epoch 17/10000
10/10 - 0s - loss: 8.7343 - v

Epoch 137/10000
10/10 - 0s - loss: 8.6521 - val_loss: 8.6439
Epoch 138/10000
10/10 - 0s - loss: 8.6521 - val_loss: 8.6438
Epoch 139/10000
10/10 - 0s - loss: 8.6520 - val_loss: 8.6438
Epoch 140/10000
10/10 - 0s - loss: 8.6520 - val_loss: 8.6438
Epoch 141/10000
10/10 - 0s - loss: 8.6519 - val_loss: 8.6437
Epoch 142/10000
10/10 - 0s - loss: 8.6519 - val_loss: 8.6437
Epoch 143/10000
10/10 - 0s - loss: 8.6518 - val_loss: 8.6437
Epoch 144/10000
10/10 - 0s - loss: 8.6518 - val_loss: 8.6436
Epoch 145/10000
10/10 - 0s - loss: 8.6518 - val_loss: 8.6436
Epoch 146/10000
10/10 - 0s - loss: 8.6517 - val_loss: 8.6436
Epoch 147/10000
10/10 - 0s - loss: 8.6517 - val_loss: 8.6436
Epoch 148/10000
10/10 - 0s - loss: 8.6517 - val_loss: 8.6435
Epoch 149/10000
10/10 - 0s - loss: 8.6516 - val_loss: 8.6435
Epoch 150/10000
10/10 - 0s - loss: 8.6516 - val_loss: 8.6435
Epoch 151/10000
10/10 - 0s - loss: 8.6516 - val_loss: 8.6434
Epoch 152/10000
10/10 - 0s - loss: 8.6515 - val_loss: 8.6434
Epoch 153/10000
10/10 - 

Epoch 272/10000
10/10 - 0s - loss: 8.6458 - val_loss: 8.6390
Epoch 273/10000
10/10 - 0s - loss: 8.6458 - val_loss: 8.6390
Epoch 274/10000
10/10 - 0s - loss: 8.6458 - val_loss: 8.6390
Epoch 275/10000
10/10 - 0s - loss: 8.6457 - val_loss: 8.6390
Epoch 276/10000
10/10 - 0s - loss: 8.6457 - val_loss: 8.6390
Epoch 277/10000
10/10 - 0s - loss: 8.6457 - val_loss: 8.6389
Epoch 278/10000
10/10 - 0s - loss: 8.6457 - val_loss: 8.6389
Epoch 279/10000
10/10 - 0s - loss: 8.6456 - val_loss: 8.6389
Epoch 280/10000
10/10 - 0s - loss: 8.6456 - val_loss: 8.6389
Epoch 281/10000
10/10 - 0s - loss: 8.6456 - val_loss: 8.6389
Epoch 282/10000
10/10 - 0s - loss: 8.6456 - val_loss: 8.6389
Epoch 283/10000
10/10 - 0s - loss: 8.6456 - val_loss: 8.6389
Epoch 284/10000
10/10 - 0s - loss: 8.6456 - val_loss: 8.6389
Epoch 285/10000
10/10 - 0s - loss: 8.6456 - val_loss: 8.6389
Epoch 286/10000
10/10 - 0s - loss: 8.6456 - val_loss: 8.6388
Epoch 287/10000
10/10 - 0s - loss: 8.6455 - val_loss: 8.6388
Epoch 288/10000
10/10 - 

Epoch 407/10000
10/10 - 0s - loss: 8.6278 - val_loss: 8.6256
Epoch 408/10000
10/10 - 0s - loss: 8.6273 - val_loss: 8.6252
Epoch 409/10000
10/10 - 0s - loss: 8.6265 - val_loss: 8.6245
Epoch 410/10000
10/10 - 0s - loss: 8.6258 - val_loss: 8.6241
Epoch 411/10000
10/10 - 0s - loss: 8.6252 - val_loss: 8.6237
Epoch 412/10000
10/10 - 0s - loss: 8.6247 - val_loss: 8.6233
Epoch 413/10000
10/10 - 0s - loss: 8.6241 - val_loss: 8.6228
Epoch 414/10000
10/10 - 0s - loss: 8.6236 - val_loss: 8.6225
Epoch 415/10000
10/10 - 0s - loss: 8.6231 - val_loss: 8.6220
Epoch 416/10000
10/10 - 0s - loss: 8.6225 - val_loss: 8.6217
Epoch 417/10000
10/10 - 0s - loss: 8.6220 - val_loss: 8.6213
Epoch 418/10000
10/10 - 0s - loss: 8.6216 - val_loss: 8.6209
Epoch 419/10000
10/10 - 0s - loss: 8.6211 - val_loss: 8.6206
Epoch 420/10000
10/10 - 0s - loss: 8.6206 - val_loss: 8.6202
Epoch 421/10000
10/10 - 0s - loss: 8.6202 - val_loss: 8.6199
Epoch 422/10000
10/10 - 0s - loss: 8.6197 - val_loss: 8.6195
Epoch 423/10000
10/10 - 

Epoch 542/10000
10/10 - 0s - loss: 8.6072 - val_loss: 8.6077
Epoch 543/10000
10/10 - 0s - loss: 8.6072 - val_loss: 8.6077
Epoch 544/10000
10/10 - 0s - loss: 8.6072 - val_loss: 8.6077
Epoch 545/10000
10/10 - 0s - loss: 8.6071 - val_loss: 8.6076
Epoch 546/10000
10/10 - 0s - loss: 8.6071 - val_loss: 8.6076
Epoch 547/10000
10/10 - 0s - loss: 8.6071 - val_loss: 8.6076
Epoch 548/10000
10/10 - 0s - loss: 8.6071 - val_loss: 8.6076
Epoch 549/10000
10/10 - 0s - loss: 8.6071 - val_loss: 8.6075
Epoch 550/10000
10/10 - 0s - loss: 8.6070 - val_loss: 8.6075
Epoch 551/10000
10/10 - 0s - loss: 8.6070 - val_loss: 8.6075
Epoch 552/10000
10/10 - 0s - loss: 8.6070 - val_loss: 8.6075
Epoch 553/10000
10/10 - 0s - loss: 8.6070 - val_loss: 8.6075
Epoch 554/10000
10/10 - 0s - loss: 8.6070 - val_loss: 8.6074
Epoch 555/10000
10/10 - 0s - loss: 8.6070 - val_loss: 8.6074
Epoch 556/10000
10/10 - 0s - loss: 8.6070 - val_loss: 8.6074
Epoch 557/10000
10/10 - 0s - loss: 8.6069 - val_loss: 8.6074
Epoch 558/10000
10/10 - 

Epoch 677/10000
10/10 - 0s - loss: 8.6059 - val_loss: 8.6059
Epoch 678/10000
10/10 - 0s - loss: 8.6059 - val_loss: 8.6059
Epoch 679/10000
10/10 - 0s - loss: 8.6059 - val_loss: 8.6059
Epoch 680/10000
10/10 - 0s - loss: 8.6059 - val_loss: 8.6059
Epoch 681/10000
10/10 - 0s - loss: 8.6059 - val_loss: 8.6059
Epoch 682/10000
10/10 - 0s - loss: 8.6059 - val_loss: 8.6059
Epoch 683/10000
10/10 - 0s - loss: 8.6059 - val_loss: 8.6059
Epoch 684/10000
10/10 - 0s - loss: 8.6059 - val_loss: 8.6059
Epoch 685/10000
10/10 - 0s - loss: 8.6059 - val_loss: 8.6059
Epoch 686/10000
10/10 - 0s - loss: 8.6059 - val_loss: 8.6059
Epoch 687/10000
10/10 - 0s - loss: 8.6059 - val_loss: 8.6059
Epoch 688/10000
10/10 - 0s - loss: 8.6058 - val_loss: 8.6059
Epoch 689/10000
10/10 - 0s - loss: 8.6058 - val_loss: 8.6059
Epoch 690/10000
10/10 - 0s - loss: 8.6058 - val_loss: 8.6059
Epoch 691/10000
10/10 - 0s - loss: 8.6058 - val_loss: 8.6059
Epoch 692/10000
10/10 - 0s - loss: 8.6058 - val_loss: 8.6059
Epoch 693/10000
10/10 - 

Epoch 812/10000
10/10 - 0s - loss: 8.6054 - val_loss: 8.6053
Epoch 813/10000
10/10 - 0s - loss: 8.6054 - val_loss: 8.6053
Epoch 814/10000
10/10 - 0s - loss: 8.6054 - val_loss: 8.6053
Epoch 815/10000
10/10 - 0s - loss: 8.6054 - val_loss: 8.6053
Epoch 816/10000
10/10 - 0s - loss: 8.6054 - val_loss: 8.6053
Epoch 817/10000
10/10 - 0s - loss: 8.6054 - val_loss: 8.6052
Epoch 818/10000
10/10 - 0s - loss: 8.6054 - val_loss: 8.6052
Epoch 819/10000
10/10 - 0s - loss: 8.6054 - val_loss: 8.6052
Epoch 820/10000
10/10 - 0s - loss: 8.6054 - val_loss: 8.6052
Epoch 821/10000
10/10 - 0s - loss: 8.6054 - val_loss: 8.6052
Epoch 822/10000
10/10 - 0s - loss: 8.6054 - val_loss: 8.6052
Epoch 823/10000
10/10 - 0s - loss: 8.6054 - val_loss: 8.6052
Epoch 824/10000
10/10 - 0s - loss: 8.6054 - val_loss: 8.6052
Epoch 825/10000
10/10 - 0s - loss: 8.6054 - val_loss: 8.6053
Epoch 826/10000
10/10 - 0s - loss: 8.6054 - val_loss: 8.6052
Epoch 827/10000
10/10 - 0s - loss: 8.6054 - val_loss: 8.6052
Epoch 828/10000
10/10 - 

In [ ]:
a = model.predict(data["input_features"])
sns.distplot(a[:,0])
sns.distplot(a[:,1])